# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [2]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# A for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Using glob to join the file path and roots with the subdirectories
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [6]:
# read in csv file to understand the dataset
df = pd.read_csv('event_datafile_new.csv')
df.head()
df.info()
df[df['song']=='All Hands Against His Own']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2792,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5135,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6298,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29


# Part II. Completing the Apache Cassandra coding portion of the project. 

##  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
 
***`SELECT artist, song, length FROM music_history WHERE sessionId = 338 AND itemInSession = 4;`***


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
***`SELECT artist, song, firstName, lastName FROM song_by_user_session WHERE userId = 10 AND sessionId = 182;`***


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

***`SELECT firstName, lastName FROM song_by_user WHERE song = 'All Hands Against His Own' AND userId IN (80, 95, 29);`***

#### <font color=red>**Creating table 1 and insert records with respect to query 1**</font>

*Query_1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4*

***since the query is asking for artist, song and length with primary access columns; sessionId and itemInSession, the table would start with the columns sessionId and itemInSession (partition keys). The columns artist, song and length become the cluster columns to ensure uniqueness of the row. We only need the above mentioned columns so the table is modelled based on these columns***

`Table Name: music_history
column 1: sessionId INT
column 2: itemInSession INT
column 3: artist TEXT
column 4: song TEXT
column 5: length FLOAT
PRIMARY KEY((sessionId, itemInSession), artist, song, length)`

In [10]:
#table 1
query = "CREATE TABLE IF NOT EXISTS music_history"
query = query + "(sessionId INT, itemInSession INT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY((sessionId, itemInSession), artist, song, length))"
try:
    session.execute(query)
except Exception as e:
    print(e)              

#### Inserting data into table

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT statements
        query = "INSERT INTO music_history (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Verifying that the data have been inserted into the table or validating the model

In [12]:
query = "SELECT artist, song, length FROM music_history WHERE sessionId = 338 AND itemInSession = 4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### CREATING TABLES AND INSERT RECORDS FOR EACH OF THE REMAINING QUERIES

#### <font color= red>Table 2 with respect to query 2 </font>

Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

***In this table, itemInSession is the cluster column since it has been stated in the query. The table starts with the partition keys; userId and sessionId followed by the columns in the select statement above.***

`Table Name: song_by_user_session
column 1: userId INT
column 2: sessionId INT
column 3: itemInSession INT
column 4: artist TEXT
column 5: song TEXT
column 6: firstName VARCHAR
column 7: lastName VARCHAR
PRIMARY KEY((userId, sessionId), itemInSession)`

In [13]:
# table 2
query = "CREATE TABLE IF NOT EXISTS song_by_user_session"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName varchar, lastName varchar, PRIMARY KEY((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                  

#### Inserting data into table

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#INSERT statements
        query = "INSERT INTO song_by_user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Validating the model

In [15]:
# Adding in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, firstName, lastName FROM song_by_user_session WHERE userId = 10 AND sessionId = 182;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### <font color=red>Table 3 with respect to query 3</font>

*Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'*

***This table has four columns with the partition keys: song, and userId to ensure the uniqueness of the row and cluster columns to sort by: firstName, lastName. Again, the table is modelled around the requested columns in the query***

`Table Name: song_by_user
column 1: song TEXT
column 2: userId INT
column 3: firstName VARCHAR
column 4: lastName VARCHAR
PRIMARY KEY((song, userId), firstName, lastName)`

In [16]:
# table 3
query = "CREATE TABLE IF NOT EXISTS song_by_user"
query = query + "(song TEXT, userId INT, firstName VARCHAR, lastName VARCHAR, PRIMARY KEY((song, userId), firstName, lastName))"
try:
    session.execute(query)
except Exception as e:
    print(e)            

#### Inserting data into table

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#INSERT statements
        query = "INSERT INTO song_by_user (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Validating the model

In [18]:
# Adding in the SELECT statement to verify the data was entered into the table
query = "SELECT firstName, lastName FROM song_by_user WHERE song = 'All Hands Against His Own' AND userId IN (80, 95, 29)"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping the tables before closing out the sessions

In [19]:
query = "DROP TABLE music_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE song_by_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE song_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Closing the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()